# Fetch News articles from November 15, 2019 to December 17, 2023.

The data is extracted by looking for the following websites:

- El Pais


In [ ]:
import requests
from bs4 import BeautifulSoup
import json
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time

# 1. El País
## 1.1. Extract Pages


In [ ]:
def verify_data(data):
	try:
		data = data.text
		return data
	except:
		return "None"

def get_pages(term, document_path):

	search_term = term
	search_term = search_term.replace(' ', '+')  
	num_pages = 1
	json_pages_info = {
		"pages": []
	}

	headers = {'User-Agent': 'Chrome/58.0.3029.110'}

	all_pages_links = []

	for page in range(0, num_pages):
		url = f'https://elpais.com/buscador/Estallido%20Social%20en%20Chile/{page}'
		response = requests.get(url, headers=headers)

		if response.status_code == 200:
			print("status 200")

			response.encoding = 'ISO-8859-1'
			content = response.content.decode('ISO-8859-1')
			soup = BeautifulSoup(content, 'html.parser')
			
			with open('output.html', 'w', encoding='utf-8') as f:
				f.write(soup.prettify())

			news = soup.find("div", class_="bu_b _g-o").find_all('article')

			for page_section in news:

				timestamp = (int)(page_section["data-timestamp"])
				if (timestamp < 1573786800 or timestamp > 1702782000):
					print("La noticia no corresponde a la fecha solicitada")
					continue

				title = verify_data(page_section.find("header").find("h2"))
				category = verify_data(page_section.find("header").find("a"))
				description = verify_data(page_section.find("p", class_="c_d"))
				date = verify_data(page_section.find("div", class_="c_f"))
				image_link = page_section.find("figure").find("a", href=True)["href"]
				author = verify_data(page_section.find("div", class_="c_a"))

				link_section = page_section.find("header").find("h2").find('a', href=True)
				link = link_section['href']

				if (all_pages_links.count(link) > 0):
					print("Sitio ya ingresado")
					continue
				else:
					all_pages_links.append(link)

				link_info = {
					"newscast" : "El País",
					"title": title,
					"description": description,
					"category": category,
					"date": date,
					"image_link": image_link,
					"author": author,
					"link": link,
				}
				json_pages_info["pages"].append(link_info)
		else:
			print(f"Error al acceder a la página {page + 1}: {response.status_code}")
			
	print(f"Se han encontrado: {len(json_pages_info['pages'])} páginas")

	with open(f"{document_path}", 'w', encoding='utf-8') as file:
		json.dump(json_pages_info, file, ensure_ascii=False, indent=4)

In [ ]:
get_pages('Estallido Social Chile', './pages_extracted/elpais_pages.json')

## 1.2 El Pais - Extract content from each page

In [ ]:
def get_page_content(page, url):

	page["page_content"] = {}

	headers = {'User-Agent': 'Chrome/58.0.3029.110'}

	response = requests.get(url, headers=headers)

	if response.status_code == 200:

		page_content = {}
		print("status 200")

		response.encoding = 'utf-8'
		content = response.content.decode('utf-8')
		soup = BeautifulSoup(content, 'html.parser')
		
		with open('output.html', 'w', encoding='utf-8') as f:
			f.write(soup.prettify())

		print("Extracción de datos...")

		text = ""

		title = soup.find("h1", class_="a_t")
		content = soup.find("div", class_="a_c clearfix")
		paragraph = content.find_all("p") if content else None
		for p in paragraph:
			text += p.text.strip() + "\n"

		page_content = {
			"header": title.text.strip() if title else "No header found",
			"content": text.strip() if text else "No content found",
		}

		page["page_content"] = page_content
	else:
		print(f"Error al acceder al contenido de la página {url}: {response.status_code}")
			
	# print(f"Se han encontrado: {len(json_data['content'])} páginas")

	# with open(f"{document_path}", 'w', encoding='utf-8') as file:
	# 	json.dump(json_data, file, ensure_ascii=False, indent=4)

In [ ]:
# original_json_data = {}
# data_decoded = {"pages": []}

# with open("pages_extracted/elpais_pages.json", 'r', encoding='latin1') as file:
#     original_json_data = json.load(file)
#     print(len(original_json_data["pages"]))

# for page in original_json_data["pages"]:
# 	page["newscast"] = page["newscast"].encode("latin1").decode("utf-8")
# 	page["title"] = page["title"].encode("latin1").decode("utf-8")
# 	page["description"] = page["description"].encode("latin1").decode("utf-8")
# 	page["category"] = page["category"].encode("latin1").decode("utf-8")
# 	page["date"] = page["date"].encode("latin1").decode("utf-8")
# 	page["image_link"] = page["image_link"].encode("latin1").decode("utf-8")
# 	page["author"] = page["author"].encode("latin1").decode("utf-8")
# 	page["link"] = page["link"].encode("latin1").decode("utf-8")
# 	data_decoded["pages"].append(page)
      
# with open("data/test.json", 'w', encoding='utf-8') as file:
# 	json.dump(data_decoded, file, ensure_ascii=False, indent=4)

In [ ]:
original_json_data = {}
json_data_with_content ={"pages": []}

with open("pages_extracted/elpais_pages.json", 'r', encoding='utf-8') as file:
    original_json_data = json.load(file)
    print(len(original_json_data["pages"]))
    
for page in original_json_data["pages"]:
    link = page["link"]
    get_page_content(page, link)
    json_data_with_content["pages"].append(page)
	
with open("data/test.json", 'w', encoding='utf-8') as file:
	json.dump(json_data_with_content, file, ensure_ascii=False, indent=4)

# The Clinic

In [ ]:

def verify_data(data):
	try:
		data = data.text
		return data
	except:
		return "None"

def get_pages(document_path):

	num_pages = 1
	json_pages_info = {
		"pages": []
	}

	headers = {'User-Agent': 'Chrome/58.0.3029.110'}

	all_pages_links = []

	for page in range(0, num_pages):
		url = f'https://www.theclinic.cl/busqueda/?q=estallido+social'
		response = requests.get(url, headers=headers)

		if response.status_code == 200:
			print("status 200")

			response.encoding = 'utf-8'
			# content = response.content.decode('utf-8')
			soup = BeautifulSoup(response.text, 'html.parser')
			
			with open('output.html', 'w', encoding='utf-8') as f:
				f.write(soup.prettify())

			news = soup.find("div", class_="gsc-expansionArea").find_all("div", class_='gsc-webResult gsc-result')

			for page_section in news:
				print("encontrado")

				# timestamp = (int)(page_section["data-timestamp"])
				# if (timestamp < 1573786800 or timestamp > 1702782000):
				# 	print("La noticia no corresponde a la fecha solicitada")
				# 	continue

				# title = verify_data(page_section.find("header").find("h2"))
				# category = verify_data(page_section.find("header").find("a"))
				# description = verify_data(page_section.find("p", class_="c_d"))
				# date = verify_data(page_section.find("div", class_="c_f"))
				# image_link = page_section.find("figure").find("a", href=True)["href"]
				# author = verify_data(page_section.find("div", class_="c_a"))

				# link_section = page_section.find("header").find("h2").find('a', href=True)
				# link = link_section['href']

				# if (all_pages_links.count(link) > 0):
				# 	print("Sitio ya ingresado")
				# 	continue
				# else:
				# 	all_pages_links.append(link)

				# link_info = {
				# 	"newscast" : "El País",
				# 	"title": title,
				# 	"description": description,
				# 	"category": category,
				# 	"date": date,
				# 	"image_link": image_link,
				# 	"author": author,
				# 	"link": link,
				# }
				# json_pages_info["pages"].append(link_info)
		else:
			print(f"Error al acceder a la página {page + 1}: {response.status_code}")
			
	print(f"Se han encontrado: {len(json_pages_info['pages'])} páginas")

	with open(f"{document_path}", 'w', encoding='utf-8') as file:
		json.dump(json_pages_info, file, ensure_ascii=False, indent=4)

In [ ]:
get_pages('./pages_extracted/theClinic_pages.json')

## CNN en Español


In [ ]:

def verify_data(data):
	try:
		data = data.text
		return data
	except:
		return "None"

def get_pages(document_path):

	num_pages = 1
	json_pages_info = {
		"pages": []
	}

	headers = {'User-Agent': 'Chrome/58.0.3029.110'}

	all_pages_links = []

	for page in range(0, num_pages):
		url = f"https://cnnespanol.cnn.com/search?q=Estallido+Social+Chile&from=0&size=10&page={page}&sort=newest&types=all&section="
		response = requests.get(url, headers=headers)

		if response.status_code == 200:
			print("status 200")

			response.encoding = 'utf-8'
			soup = BeautifulSoup(response.text, 'html.parser')
			
			with open('output.html', 'w', encoding='utf-8') as f:
				f.write(soup.prettify())

			# news = soup.find("div", class_="container__field-links container_list-images-with-description__field-links").find_all("div", class_="card2")

			# for page_section in news:
			# 	print("encontrado")

				# timestamp = (int)(page_section["data-timestamp"])
				# if (timestamp < 1573786800 or timestamp > 1702782000):
				# 	print("La noticia no corresponde a la fecha solicitada")
				# 	continue

				# title = verify_data(page_section.find("header").find("h2"))
				# category = verify_data(page_section.find("header").find("a"))
				# description = verify_data(page_section.find("p", class_="c_d"))
				# date = verify_data(page_section.find("div", class_="c_f"))
				# image_link = page_section.find("figure").find("a", href=True)["href"]
				# author = verify_data(page_section.find("div", class_="c_a"))

				# link_section = page_section.find("header").find("h2").find('a', href=True)
				# link = link_section['href']

				# if (all_pages_links.count(link) > 0):
				# 	print("Sitio ya ingresado")
				# 	continue
				# else:
				# 	all_pages_links.append(link)

				# link_info = {
				# 	"newscast" : "El País",
				# 	"title": title,
				# 	"description": description,
				# 	"category": category,
				# 	"date": date,
				# 	"image_link": image_link,
				# 	"author": author,
				# 	"link": link,
				# }
				# json_pages_info["pages"].append(link_info)
		else:
			print(f"Error al acceder a la página {page + 1}: {response.status_code}")
			
	print(f"Se han encontrado: {len(json_pages_info['pages'])} páginas")

	with open(f"{document_path}", 'w', encoding='utf-8') as file:
		json.dump(json_pages_info, file, ensure_ascii=False, indent=4)

In [ ]:
get_pages('./pages_extracted/cnnEspañol_pages.json')

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time

# Configurar opciones para Chrome (modo headless)
options = Options()
options.add_argument("--headless") 
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")

for page in range(1, 3):
	print(page)

	driver = webdriver.Chrome(options=options)

	from_value = (page - 1) * 10
	url = f"https://cnnespanol.cnn.com/search?q=Estallido+Social+Chile&from={from_value}&size=10&sort=newest&types=all&section="

	driver.get(url)

	# Esperar a que la página cargue completamente
	time.sleep(3)

	with open(f"output{page}.html", "w", encoding="utf-8") as f:
			f.write(driver.page_source)
	driver.quit()


## La Tercera


In [ ]:

json_pages_info = {"pages": []}

# Configurar opciones para Chrome (modo headless)
options = Options()
options.add_argument("--headless") 
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")

driver = webdriver.Chrome(options=options)

# from_value = (page - 1) * 10
url = f"https://www.latercera.com/search/?q=Estallido%20Social%20Chile"

driver.get(url)

# Esperar a que la página cargue completamente
time.sleep(3)

elements = driver.find_element(By.CLASS_NAME, "gsc-expansionArea")
articles = elements.find_elements(By.CLASS_NAME, "gsc-webResult")

print(f"Se han encontrado: {len(articles)} páginas")


for article in articles:
	print("encontrado")
	title = article.find_element(By.CLASS_NAME, "gs-title").text
	description = article.find_element(By.CLASS_NAME, "gs-snippet").text
	date = description[0:11]
	link = article.find_element(By.CLASS_NAME, "gs-title").find_element(By.CLASS_NAME, "gs-title").get_attribute("href")

	link_info = {
		"newscast" : "La Tercera",
		"title": title,
		"description": description,
		"date": date,
		"link": link,
	}

	# print(f"Title: {title}")
	# print(f"Description: {description}")
	# print(f"Date: {date}")
	# print(f"Link: {link}")
	json_pages_info["pages"].append(link_info)


pages_list = driver.find_element(By.CLASS_NAME, "gsc-cursor")
pages_links = pages_list.find_elements(By.CLASS_NAME, "gsc-cursor-page")

print(f"Se han encontrado: {len(pages_links)} links")

pages_links[0].click()

# Esperar a que la página cargue completamente
time.sleep(3)

elements = driver.find_element(By.CLASS_NAME, "gsc-expansionArea")
articles = elements.find_element(By.CLASS_NAME, "gsc-webResult")

print(f"Se han encontrado: {len(articles)} páginas")

with open(f"output.html", "w", encoding="utf-8") as f:
		f.write(driver.page_source)

driver.quit()


with open(f"./pages_extracted/laTercera_pages.json", 'w', encoding='utf-8') as file:
	json.dump(json_pages_info, file, ensure_ascii=False, indent=4)


In [ ]:
import requests
import json
import re


url = "https://cse.google.com/cse/element/v1?rsz=filtered_cse&num=10&hl=es&source=gcsc&start=30&cselibv=75c56d121cde450a&cx=0099769ad8b6d3d5e&q=Estallido+Social+Chile&safe=off&cse_tok=AB-tC_6ZjKQAJAOQmh9snYkP7ics%3A1746614694780&exp=cc%2Capo&callback=google.search.cse.api6184&rurl=https%3A%2F%2Fwww.latercera.com%2Fsearch%2F%3Fq%3DEstallido%2520Social%2520Chile"
response = requests.get(url)

if response.status_code == 200:
    match = re.search(r'^[^(]+\((.*)\);?$', response.text)
    if match:
        json_str = match.group(1)
        data = json.loads(json_str)
        print(data)
    else:
        print("No se pudo extraer el JSON.")